In [1]:
#Import bibliotek
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import pyodbc 

In [ ]:
#Zaczytanie pliku źródłowego do tabeli źródłowej
#df_raw = pd.read_excel(r"C:\01 PRZENIESIENIE\KPEC BYDGOSZCZ\01 WDROŻENIE\01 DANE\02 KALIBRACJA\DANE OBROBIONE\K-501.xlsx",engine="openpyxl")



#Definicja kolumn w tabeli docelowej
kolumny1 = ["TagID","TagTime","TagValue"]

#Utworzenie tabeli docelowej z kolumnami docelowymi
df1 = pd.DataFrame(columns = kolumny1)

#Określenie ścieżki dostępu do folderu z plikami źródłowymi
directory = os.path.join(r"C:\01 PRZENIESIENIE\KPEC BYDGOSZCZ\01 WDROŻENIE\01 DANE\02 KALIBRACJA\DANE OBROBIONE 23022022")
print("!!! ZACZYTYWANIE DANYCH !!!")
for root,dirs,files in os.walk(directory):
    
    for file in files:
       
        
        if file.endswith(".xlsx"):
            print(file)
            
            #Zaczytanie pliku źródłowego do tabeli źródłowej
            df_raw = pd.read_excel(f"C:\\01 PRZENIESIENIE\\KPEC BYDGOSZCZ\\01 WDROŻENIE\\01 DANE\\02 KALIBRACJA\\DANE OBROBIONE 23022022\\{file}",
                                   engine="openpyxl")



            #Zaczytanie kolumn z tabeli źródłowej
            kolumny = df_raw.columns

            #print(kolumny)
            #Usunięcie pierwszego wiersza z tabeli źródłowej
            df = df_raw.drop(index = 0)



            #Przeniesienie wartości z chwilowej tabeli źródełowej to tabeli docelowej
            for i in range(1,len(kolumny)):

                #Zaczytanie kolumny z czasem oraz wartością parametru
                df_new = df[[kolumny[0],kolumny[i]]]

                #Zdefiniowanie nazw chwilowej tabeli docelowej
                df_new.columns = ["TagTime", "TagValue"]

                #Dodanie kolumny do tabeli chwilowej z nazwą parametru
                df_new["TagID"] = kolumny[i]

                #Edycja kolejności kolumn tabeli
                cols = df_new.columns.tolist()
                cols = cols[-1:] + cols[:-1]
                df_new = df_new[cols]


                #Dodanie chwilowej tabeli docelowej do tabeli docelowej
                df1 = df1.append(df_new, ignore_index = True)

print("!!! CZYSZCZENIE DANYCH !!!")                
#Usunięcie zbędnych zmiennych              
mask = df1['TagID'].str.contains('Unn', case=False, na=False)
df1 = df1[~mask]
df1.reset_index(drop = True, inplace = True)


mask2 = df1['TagID'].str.contains("[Z][0-9][a-z]|[Z][0-9]", case=False, na=False)
df1 = df1[~mask2]


#Usunięcie zmiennych z PS              
mask2 = df1['TagID'].str.contains('PS', case=False, na=False)
df1 = df1[~mask2]
df1.reset_index(drop = True, inplace = True)

#Zamiana wartości Null na 0
df1['TagValue'].replace("(null)",0, inplace = True)

#Zmiana wartości NaN na 0
df1.replace(np.nan ,0, inplace = True)

#Konwersja typu TagValue do numerycznej
df1["TagValue"] = pd.to_numeric(df1["TagValue"])


print("!!! PODSUMOWANIE DANYCH !!!") 
#Podsumowanie danych - brakow, min, max, mean

df_metryka = pd.DataFrame()
parametry = df1["TagID"].unique()

for i in parametry:
    
    df_stat = df1.loc[df1["TagID"] == i]

    max = df_stat['TagValue'].max()
    min = df_stat['TagValue'].min()
    mean = df_stat['TagValue'].mean()

    
    liczba_zer = (df_stat.TagValue.values == 0).sum()
    index = df_stat.index
    liczba_wierszy = len(index) 

    procent_zer = round(liczba_zer * 100 / liczba_wierszy,2)
    

    D = {"1. ID": i, "2. Minimum": round(min,2), "3. Srednia": round(mean,2), "4. Maksimum": round(max,2), "5. Procent brakow": procent_zer}
    df_metryka = df_metryka.append(D, ignore_index = True)
df_metryka.to_csv("C:\\01 PRZENIESIENIE\\KPEC BYDGOSZCZ\\01 WDROŻENIE\\01 DANE\\02 KALIBRACJA\\Podsumowanie_danych.csv", index = False, encoding = 'utf-8', decimal = "," )

print("!!! EKSPORT DANYCH DO SQL !!!")
"""#Eksport danych do SQL

connStr = pyodbc.connect('Driver={SQL Server};'
                            'Server=KPEC-KELVIN;'
                            'Database=Kalibracja;'
                            'Trusted_Connection=yes;')
cursor = connStr.cursor()

for index,row in df1.iterrows():
    
    insert_stmt = (
        "INSERT INTO dbo.Dane([TagID],[TagTime],[TagValue])"
        "VALUES (?,?,?)" )
        
    cursor.execute(insert_stmt,tuple(row)) 
                         
    connStr.commit()
                         
cursor.close()
connStr.close()"""

In [ ]:
#----------------WYKRES 1-------------------

fig1, ax1 = plt.subplots()
ax2 = ax1.twinx()

lns1 = ax1.plot(x, y_p,label="Moc pomiarowa, W", color = "orange",linewidth=2)
lns2 = ax1.plot(x, y_m,label="Moc modelowa, W",linewidth=2)
lns3 = ax2.plot(x, y_t,label="Temperatura otoczenia, $^\circ$C", color = "red",linestyle='dashed',linewidth=1 )
lns4 = ax2.plot(x, y_w,label="Prędkość wiatru m/s", color = "black",linestyle='dashed' ,linewidth=1)

 
fig1.set_figheight(wysokosc)
fig1.set_figwidth(szerokosc)
plt.xticks(range(0,24))

ax1.set_xlabel("Godzina doby")
ax1.set_ylabel("Moc, W")
#ax1.tick_params(axis="y")

ax2.set_ylabel("Tempertura $^\circ$C; Wiatr, m/s")
ax2.tick_params(axis="y")
lns_3 = lns1+lns2+lns3+lns4
labs_3 = [l.get_label() for l in lns_3]
ax1.legend(lns_3, labs_3, loc=0)